In [ ]:
import json, os
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.decomposition import PCA
# from wpca import WPCA
from sklearn.preprocessing import StandardScaler
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
from aquabyte.visualize import Visualizer, _normalize_world_keypoints
from aquabyte.template_matching import find_matches_and_homography
import random
import pickle
import cv2
from copy import copy
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
from multiprocessing import Pool, Manager
import matplotlib.cm as cm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

<h1> Extract base data from database </h1>

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))
query = """
    SELECT * FROM prod.crop_annotation ca
    INNER JOIN prod.annotation_state pas on pas.id=ca.annotation_state_id
    WHERE ca.service_id = (SELECT ID FROM prod.service where name='LATI')
    AND ca.pen_id = 56
    AND ca.captured_at >= '2019-10-25' AND ca.captured_at < '2019-11-01'
    AND ca.annotation_state_id=7;
"""
df = rds_access_utils.extract_from_database(query)

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')


<h1> Generate Sample Template Matching Results </h1>

In [ ]:
left_image_url, right_image_url = df.left_crop_url.iloc[0], df.right_crop_url.iloc[0]
left_crop_metadata, right_crop_metadata = df.left_crop_metadata.iloc[0], df.right_crop_metadata.iloc[0]
cm = df.camera_metadata.iloc[0]

left_image_f, _, _ = s3_access_utils.download_from_url(left_image_url)
right_image_f, _, _ = s3_access_utils.download_from_url(right_image_url)
imageL = cv2.imread(left_image_f)
imageR = cv2.imread(right_image_f)
H, kps = find_matches_and_homography(imageL, imageR, cm, left_crop_metadata, right_crop_metadata)

In [ ]:
def get_wkps(kps, cm): 
    # Draw lines between matches.  Make sure to offset kp coords in second image appropriately.
    wkps = []
    for kp in kps:
        p1_x_frame, p1_y_frame, p2_x_frame, p2_y_frame = kp
        disp = abs(p1_x_frame - p2_x_frame)
        depth = depth_from_disp(disp, cm)
        wkp = convert_to_world_point(p1_x_frame, p1_y_frame, depth, cm)
        wkps.append(list(wkp))
        
    return wkps


In [ ]:
body_wkps = np.array(get_wkps(kps, cm))

<h1> Visualize World Keypoints </h1>

In [ ]:
# create local util functions for body world keypoint visualizations (will override original imports)

def _generate_rotation_matrix(u_base, v):
    u = v / np.linalg.norm(v)
    n = np.cross(u_base, u)
    n = n / np.linalg.norm(n)
    theta = -np.arccos(np.dot(u, u_base))

    R = np.array([[
        np.cos(theta) + n[0]**2*(1-np.cos(theta)), 
        n[0]*n[1]*(1-np.cos(theta)) - n[2]*np.sin(theta),
        n[0]*n[2]*(1-np.cos(theta)) + n[1]*np.sin(theta)
    ], [
        n[1]*n[0]*(1-np.cos(theta)) + n[2]*np.sin(theta),
        np.cos(theta) + n[1]**2*(1-np.cos(theta)),
        n[1]*n[2]*(1-np.cos(theta)) - n[0]*np.sin(theta),
    ], [
        n[2]*n[0]*(1-np.cos(theta)) - n[1]*np.sin(theta),
        n[2]*n[1]*(1-np.cos(theta)) + n[0]*np.sin(theta),
        np.cos(theta) + n[2]**2*(1-np.cos(theta))
    ]])
    
    return R

def _normalize_world_keypoints(wkps, rotate=True):
    body_parts = wkps.keys()
    
    # translate keypoints such that tail notch is at origin
    translated_wkps = {bp: wkps[bp] - wkps['HYPURAL_PLATE'] for bp in body_parts}

    if not rotate:
        return translated_wkps
    
    # perform first rotation
    u_base=np.array([1, 0, 0])
    v = translated_wkps['UPPER_LIP']
    R = _generate_rotation_matrix(u_base, v)
    norm_wkps_intermediate = {bp: np.dot(R, translated_wkps[bp].T) for bp in body_parts}
    
    # perform second rotation
    u_base = np.array([0, 0, 1])
    v = norm_wkps_intermediate['ADIPOSE_FIN'] - np.array([norm_wkps_intermediate['ADIPOSE_FIN'][0], 0, 0])
    R = _generate_rotation_matrix(u_base, v)
    norm_wkps = {bp: np.dot(R, norm_wkps_intermediate[bp]) for bp in body_parts}
    
    # perform reflecton if necessary
    if norm_wkps['PECTORAL_FIN'][1] > 0:
        norm_wkps = {bp: np.array([
            norm_wkps[bp][0],
            -norm_wkps[bp][1],
            norm_wkps[bp][2]
        ]) for bp in body_parts}
    
    return norm_wkps

In [ ]:
wkps = {}
wkps['BODY'] = np.array(body_wkps)



In [ ]:
%matplotlib notebook
body_parts = [k for k in norm_wkps.keys() if k != 'BODY']
xs = [norm_wkps[bp][0] for bp in body_parts]
ys = [norm_wkps[bp][1] for bp in body_parts]
zs = [norm_wkps[bp][2] for bp in body_parts]
xs.extend(list(norm_wkps['BODY'][0]))
ys.extend(list(norm_wkps['BODY'][1]))
zs.extend(list(norm_wkps['BODY'][2]))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim3d(0, max(xs))
ax.set_ylim3d(-0.3, 0.3)
ax.set_zlim3d(-0.3, 0.3)
ax.scatter(xs, ys, zs, color='blue')

In [ ]:
%matplotlib notebook
xs = list(wkps['BODY'][:, 0])
ys = list(wkps['BODY'][:, 1])
zs = list(wkps['BODY'][:, 2])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# ax.set_xlim3d(0, max(xs))
# ax.set_ylim3d(-0.3, 0.3)
# ax.set_zlim3d(-0.3, 0.3)
ax.scatter(xs, ys, zs, color='blue')

<h1> Get Vikane Focusing Distance </h1>

In [ ]:
distances_from_camera = []
row_count = 0
for idx, row in df.iterrows():
    left_image_url, right_image_url = row.left_crop_url, row.right_crop_url
    if left_image_url is not None and right_image_url is not None:
        left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata
        cm = row.camera_metadata

        left_image_f, _, _ = s3_access_utils.download_from_url(left_image_url)
        right_image_f, _, _ = s3_access_utils.download_from_url(right_image_url)
        imageL = cv2.imread(left_image_f)
        imageR = cv2.imread(right_image_f)
        H, kps = find_matches_and_homography(imageL, imageR, cm, left_crop_metadata, right_crop_metadata)
        wkps = get_wkps(kps, cm)
        dist = np.median([wkp[1] for wkp in wkps])
        distances_from_camera.append(dist)
        print('Row Count: {}, Number of Body Points: {}, Calculated Distance: {}'.format(row_count, len(kps), dist))

In [ ]:
%matplotlib inline
plt.figure(figsize=(20, 10))
plt.hist(distances_from_camera, bins=20)
plt.grid()
plt.xlabel('Distance from camera (m)')
plt.ylabel('Frequency')
plt.title('Distance distribution for accepted Vikane fish (pen_id=56)')
plt.show()

In [ ]:
distances_from_camera

In [ ]:
ps = [1/7, 3/7, 3/7]
pens = [0, 1, 2]

In [ ]:
N = 1000
e_x = ps[0]
var_x = ps[0] * (1 - ps[0])
std_x = np.sqrt(var_x)

In [ ]:
std_err = std_x / np.sqrt(N)

In [ ]:
np.array([e_x - 2.58 * std_err, e_x + 2.58 * std_err]) * N

In [ ]:
e_x * N

In [ ]:
means = []
trials = 1000
for t in range(trials):
    pen_list = []
    cum_ps = np.cumsum(ps)
    for n in range(N):
        r = np.random.uniform()
        pen = 1 if r < ps[0] else 0
        pen_list.append(pen)
    mean = np.mean(pen_list)
    means.append(mean)


In [ ]:
means = np.array(means)

In [ ]:
len(means[(means > e_x - 2.58 * std_err) & (means < e_x + 2.58 * std_err)])

In [ ]:
len(means)